# Incorporating MLP in sphinx

Starting off, we'll train a MLP on the walls street journal __wsj__ corpus. This will give us baseline scores.
We choose wsj corpus instead of open corpora because we know the accuracy (3%) we have to achieve .

**22nd May:** Literature review. output classes of MLP tied states or phone states.According to this paper by J. hinton https://www.clsp.jhu.edu/~samuel/pdfs/scarf_mlp.pdf, Tied states were used.

**23rd May:** Writing code to generate log spectra from audio files

Libraries used:<br>**pysoundfile**<br>**python_speech_features**

Pysoundfile reads audio files in flac format and returns an array
python_speech_features provides a logfbank method that takes in a linear array and returns filter bank energies

In [2]:
import numpy as np
from python_speech_features import logfbank
import soundfile as sf
import os

In [ ]:
#for making log feats 
#path to corpus
corpus= '/media/t-rex/F/wsj/wsj0';
#this file contains paths to all flac files
f = open('flac_paths.txt','w')
    
for dir_name,subdir_name,files in os.walk(corpus):
    
    for file in files:
        if file.endswith('.flac'):
            path_2_file = dir_name+'/'+file;
            #print (path_2_file)
            f.write(path_2_file+'\n')
            (s,r) =sf.read(path_2_file)
            logfeats = logfbank(s,r, winlen=0.025, winstep=0.01, nfilt=40, nfft=1024, lowfreq=250, highfreq=None, preemph=0.97);
            np.savetxt('/media/t-rex/F/wsj/wsj_feats/'+file+'.mls',logfeats)


<br>

**25th May**: Assigning labels to features

Libraries used:<br>**Tensorflow**<br>**keras**

In [24]:
#data prep
from random import randint

all_feats=[]
all_labels=[]

root = '/media/t-rex/F/wsj/';
feats_file = root+'wsj_feats';

zorp= 0
for dir_name,subdir_name,files in os.walk(feats_file):
    if zorp ==30:
        break
    for file in files:
        if zorp ==30:
            break
        if file.endswith('.mls'):
            if zorp == 30:
                break
            zorp +=1
            path_2_file = dir_name+'/'+file;
            
            #print (path_2_file)
            feat = np.loadtxt('/media/t-rex/F/wsj/wsj_feats/'+file)
            n_frames  = feat.shape[0]

            #assigning random labels 
            labels = [randint(0,2999) for i in range(n_frames)]
            all_labels+=list(labels)
            all_feats += list(feat)
            
print(len(all_feats),len(all_labels))
#test/train split
for_training = int(len(all_feats)*0.8)
Xtrain = all_feats[:for_training]
ytrain = all_labels[:for_training]
Xtest = all_feats[for_training:]
ytest = all_labels[for_training:]

#save as compressed array
np.savez(root+'dataa',Xtrain=Xtrain,ytrain=ytrain,Xtest=Xtest,ytest=ytest)



(16786, 16786)


In [ ]:
#or save as pickle

# import pickle
# pickle.dump(Xtrain,open(root+"Xtrain.p","w"))
# pickle.dump(ytrain,open(root+"ytrain.p","w"))
# pickle.dump(Xtest,open(root+"Xtest.p","w"))
# pickle.dump(ytest,open(root+"ytest.p","w"))


**May 26th**: Writing MLP code

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import os
import numpy as np
from keras.utils import to_categorical

In [26]:
data = np.load('/media/t-rex/F/wsj/dataa.npz')

In [27]:
x_train = data['Xtrain']
y_train = data['ytrain']
x_test = data['Xtest']
y_test = data['ytest']

In [32]:
print x_train.shape
print y_train.shape

(3358, 40)
(3358,)


In [29]:
y_train = to_categorical(y_train, num_classes = 3000)

In [ ]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_dim=40))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3000, activation='softmax'))
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



model.fit(x_train,y_train,epochs=10,batch_size=2000)


In [ ]:
y_test = to_categorical(y_test, num_classes = 3000)
score = model.evaluate(x_test, y_test, batch_size=2000)
print score

**May 27th**: Setting up kaldi to find tied state alignments

**May 30th**:Refactoring